# Everything you need to edit an existing assistant

In [ ]:
%run api_assistant_creation.ipynb

In [ ]:
import json
import requests

base_path= 'http://127.0.0.1:5002/api/v1'

def stream_response(r):
    r.encoding = 'utf-8'
    if r.status_code == 200:
        for line in r.iter_lines(): 
            json_data = json.loads(line.decode('utf-8'))
            if json_data['type'] == 'chunk':
                print(json_data['content'], end='')   
        
        print()   
    else: 
        print(r.text)

## 1. Update Tag

In [ ]:
r = requests.put(f"{base_path}/tag/{tag_id}", json={
    "name": "Edited Tag Name"
})
print(r.content)

## 2. Update SystemInstruction 

In [ ]:
r = requests.put(f"{base_path}/system-instruction/{system_instruction_id}", json={
    "message": "You are a pirate. Always respond like a pirate would do. Be experimental and wild."
})
print(r.content)

## 3. Update Assistant 

In [ ]:
r = requests.put(f"{base_path}/assistant/{assistant_id}", json={
    "name": "Edited assistant name",
    "description": "Edited assistant description",
    "tags": [{
      "id": tag_id
    }],
    "systemInstructionId": system_instruction_id,
    "customSystemInstruction": None,
    "greetingMessage": "Hello. I was edited by my owner.",
    "llmIds": [
      llm_id_gpt4o
    ],
    "dataSourceIds": [
      data_source_id
    ],
    "public": True
})
print(r.content)

## 4. Create new Conversation with updated data

In [ ]:
r = requests.post(f'{base_path}/assistant/{assistant_id}/conversation', json={})

conversation_id = r.json().get("id")
print("Conversation ID: " + conversation_id)

In [ ]:
# Chat to confirm you are speaking to a pirate now
r = requests.post(f'{base_path}/conversation/{conversation_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "Hey there. Tell me a poem about the atlantic ocean."
    }
  ],
  "overrides": []
})

stream_response(r)

## 5. Delete all conversations of an assistant when saving

In [ ]:
r = requests.get(f'{base_path}/assistant/{assistant_id}/conversations', json={})
print(r)
for _conversation in r.json():
    r = requests.delete(f'{base_path}/conversation/{_conversation.get("id")}')
    print(r)